# Data wrangling<a id='2_Data_wrangling'></a>

## 1.1 Introduction

blah blah

## 1.2 Imports

In [4]:
import numpy as np
import pandas as pd
import os
import re

## 1.3 Objectives

blah blah blah

## 1.4 Function Definitions

In [23]:
#import the file and read all lines as strings
def import_dat(batch_file, rd_path):
    """
    Function for importing .dat files. Each line is read in as a string and broken up based on spaces between entries.
    It is expected that there is a ';' in the first string chunk, and multiple ':' throughout.
    Column names are manually created and entered here so it only work for the Gas Sensor Array Drift Dataset at Different Concentrations Data Set
    
    Inputs:
        batch_file - (str) filename of the specfic batch data
        rd_path - (str) folder location of where the batch file is located
    Outputs:
        batch_data - (DataFrame) batch file data in an easier to read format
    """
    with open(rd_path + '\\' + batch_file, 'r') as file: #open the file as read only
        lines = file.readlines() #save the data as a list of string lines before closing

    fd = [] # initialize a list for the formatted data
    for line in lines:
        row_list = line.split() #split the line up to get the column values for a given row
        predict_vals = [float(i[i.index(':')+1:]) for i in row_list[1:]] #extract the predictor variable values
        target_vals = [row_list[0][:row_list[0].index(';')], float(row_list[0][row_list[0].index(';') + 1:])] #extract the two target variables, chemcial number & concentration
        fd.append(predict_vals + target_vals) #add the row to the formatted data list

    col_names_pat = ['DR', '|DR|', 'EMAi0.001', 'EMAi0.01', 'EMAi0.1', 'EMAd0.001', 'EMAd0.01', 'EMAd0.1'] #variable name pattern for each sensor
    col_names = [ c + '_' + str(n) for n in range(1,17) for c in col_names_pat] #use the pattern to create column names for all 16 sensors
    col_names = col_names + ['ChemicalCode', 'Concentration'] #add the two target variable names
    batch_data = pd.DataFrame(fd, columns = col_names)
    batch_data['BatchNumber'] = re.findall(r'\d+', batch_file) * len(batch_data) #add the batch number as a feature
    
    return batch_data


## 1.5 Load Raw Data

Load the data that was provided through the website and saved in the raw_data folder using the function that was created in section 1.4

In [25]:
#Get the relative local folder where the data is stored
base_fpath = os.getcwd() #the file path to the working directory of the code
rd_path = base_fpath.replace('notebooks', 'raw_data') #raw data file path

#Loop through all raw data files and compile it into a single table using the import_dat function
f_data = pd.DataFrame() #initialize an empty DataFrame to store the combined data
raw_files_list = os.listdir(rd_path) #get a list of all file names in the raw_data folder
for rd_file in raw_files_list:
    batch_data = import_dat(rd_file, rd_path) #call the import_dat function on the current file
    f_data = f_data.append(batch_data, ignore_index = True) #add the data to the combined data DataFrame

Check that all of the data was imported and is formatted correctly

In [36]:
pd.set_option('display.max_columns', None) #display all columns to ensure every thing looks right
f_data.head() #display the first 5 rows of data

,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1,EMAi0.1_1,EMAd0.001_1,EMAd0.01_1,EMAd0.1_1,DR_2,|DR|_2,EMAi0.001_2,EMAi0.01_2,EMAi0.1_2,EMAd0.001_2,EMAd0.01_2,EMAd0.1_2,DR_3,|DR|_3,EMAi0.001_3,EMAi0.01_3,EMAi0.1_3,EMAd0.001_3,EMAd0.01_3,EMAd0.1_3,DR_4,|DR|_4,EMAi0.001_4,EMAi0.01_4,EMAi0.1_4,EMAd0.001_4,EMAd0.01_4,EMAd0.1_4,DR_5,|DR|_5,EMAi0.001_5,EMAi0.01_5,EMAi0.1_5,EMAd0.001_5,EMAd0.01_5,EMAd0.1_5,DR_6,|DR|_6,EMAi0.001_6,EMAi0.01_6,EMAi0.1_6,EMAd0.001_6,EMAd0.01_6,EMAd0.1_6,DR_7,|DR|_7,EMAi0.001_7,EMAi0.01_7,EMAi0.1_7,EMAd0.001_7,EMAd0.01_7,EMAd0.1_7,DR_8,|DR|_8,EMAi0.001_8,EMAi0.01_8,EMAi0.1_8,EMAd0.001_8,EMAd0.01_8,EMAd0.1_8,DR_9,|DR|_9,EMAi0.001_9,EMAi0.01_9,EMAi0.1_9,EMAd0.001_9,EMAd0.01_9,EMAd0.1_9,DR_10,|DR|_10,EMAi0.001_10,EMAi0.01_10,EMAi0.1_10,EMAd0.001_10,EMAd0.01_10,EMAd0.1_10,DR_11,|DR|_11,EMAi0.001_11,EMAi0.01_11,EMAi0.1_11,EMAd0.001_11,EMAd0.01_11,EMAd0.1_11,DR_12,|DR|_12,EMAi0.001_12,EMAi0.01_12,EMAi0.1_12,EMAd0.001_12,EMAd0.01_12,EMAd0.1_12,DR_13,|DR|_13,EMAi0.001_13,EMAi0.01_13,EMAi0.1_13,EMAd0.001_13,EMAd0.01_13,EMAd0.1_13,DR_14,|DR|_14,EMAi0.001_14,EMAi0.01_14,EMAi0.1_14,EMAd0.001_14,EMAd0.01_14,EMAd0.1_14,DR_15,|DR|_15,EMAi0.001_15,EMAi0.01_15,EMAi0.1_15,EMAd0.001_15,EMAd0.01_15,EMAd0.1_15,DR_16,|DR|_16,EMAi0.001_16,EMAi0.01_16,EMAi0.1_16,EMAd0.001_16,EMAd0.01_16,EMAd0.1_16,ChemicalCode,Concentration,BatchNumber
0,15596.1621,1.868245,2.371604,2.803678,7.512213,-2.739388,-3.344671,-4.847512,15326.6914,1.768526,2.269085,2.713374,6.915721,-2.488324,-3.082212,-5.056975,2789.3831,2.754759,0.430440,0.649457,1.795029,-0.426662,-0.584313,-1.438976,2581.5686,2.680623,0.399746,0.605065,1.786704,-0.400115,-0.550743,-1.728611,685.3994,1.682904,0.122736,0.223703,0.584691,-0.138196,-0.236907,-0.781959,797.7738,1.742488,0.152483,0.218904,0.841862,-0.164646,-0.315720,-0.791447,3128.8489,3.605537,0.532422,0.763062,2.118983,-0.557197,-0.809953,-2.344130,3136.8778,3.555169,0.535883,0.761388,1.499244,-0.571480,-0.944425,-2.658358,13540.6738,1.765738,2.006883,2.519022,6.261430,-2.172101,-2.694967,-3.791499,13831.7539,1.746493,2.057165,2.391239,5.695234,-2.350776,-2.888766,-8.129869,3020.9191,2.819354,0.474520,0.723993,2.160130,-0.467900,-0.638167,-1.643650,2185.9741,2.949381,0.342575,0.515090,1.340477,-0.361030,-0.493482,-1.200617,862.7479,1.779291,0.165138,0.246473,1.358106,-0.187465,-0.416382,-1.058061,1059.7562,1.896047,0.198946,0.334017,0.815048,-0.204467,-0.345119,-0.969336,3357.1124,3.860647,0.580818,0.806830,1.729739,-0.619214,-1.071137,-3.037772,3037.0390,3.972203,0.527291,0.728443,1.445783,-0.545079,-0.902241,-2.654529,1,10.0,1
1,26402.0704,2.532401,5.411209,6.509906,7.658469,-4.722217,-5.817651,-7.518333,23855.7812,2.164706,4.901063,5.971392,6.978131,-4.199424,-5.221090,-7.604863,3882.9057,3.155054,0.802987,1.172245,1.570895,-0.697881,-0.944655,-1.334743,3651.0427,3.130669,0.739278,1.101765,1.428631,-0.659641,-0.915543,-1.334217,982.9490,1.933877,0.233308,0.378052,0.717916,-0.225572,-0.354591,-0.748061,1162.2378,2.039431,0.278337,0.486908,0.840494,-0.271369,-0.473164,-0.718214,4332.1543,3.990162,1.002503,1.586233,1.926608,-0.913245,-1.335612,-1.773328,4387.5284,4.011519,1.009769,1.647830,1.962858,-0.944028,-1.631468,-1.925095,20553.5645,2.108870,4.266941,5.185937,6.398693,-3.636176,-4.497039,-7.464548,22540.1933,2.222146,4.621283,5.573620,6.756039,-3.986434,-4.953663,-6.561748,4240.3011,3.262893,0.887237,1.297617,1.630074,-0.768036,-1.244472,-1.419646,3079.0621,3.399659,0.649639,0.948781,1.369550,-0.598444,-0.820868,-1.081919,1237.0131,2.067927,0.305793,0.558008,0.948564,-0.289557,-0.501078,-1.191404,1459.6356,2.149242,0.361007,0.614619,0.950204,-0.327339,-0.503861,-0.838039,4685.7753,4.277604,1.080629,1.818559,2.237534,-1.004812,-1.530519,-1.994993,4176.4453,4.281373,0.980205,1.628050,1.951172,-0.889333,-1.323505,-1.749225,1,20.0,1
2,42103.5820,3.454189,8.198175,10.508439,11.611003,-7.668313,-9.478675,-12.230939,37562.3008,2.840403,7.386357,9.511703,12.129657,-6.689464,-8.184333,-10.196169,5476.3898,3.929721,1.132118,1.717418,

The columns headers follow the correct sequence based on the meta data provided with the dataset (below), and includes the chemical and concentration information. Both the chemcial code and concentration are potential target parameters and most likely both will need to be predicted.

The BatchNumber and ChemicalCode parameters are categorical, all others are numerical.

"Thus, each feature vector contains the 8 features extracted from each particular sensor, resulting in a 128-dimensional feature vector (8 features x 16 sensors) containing all the features and organized as follows:
DR_1, |DR|_1, EMAi0.001_1, EMAi0.01_1, EMAi0.1_1, EMAd0.001_1, EMAd0.01_1, EMAd0.1_1, DR_2, |DR|_2, EMAi0.001_2, EMAi0.01_2, EMAi0.1_2, EMAd0.001_2, EMAd0.01_2, EMAd0.1_2,..., DR_16, |DR|_16, EMAi0.001_16, EMAi0.01_16, EMAi0.1_16, EMAd0.001_16, EMAd0.01_16, EMAd0.1_16
where: DR_j and |DR|_j are the R and the normalized R features, respectively. EMAi0.001_j, EMAi0.01_j, and EMAi0.1_j, are the emaR of the rising transient portion of the sensor response for Alfa 0.001, 0.01, and 0.1, respectively. EMAd0.001_j, EMAd0.01_j, and EMAd0.1_j, are emaR of the decaying transient portion of the sensor response for Alfa 0.001, 0.01, and 0.1, respectively."
-- From http://archive.ics.uci.edu/ml/datasets/Gas+Sensor+Array+Drift+Dataset+at+Different+Concentrations

## 1.6 Explore the Data

In [37]:
pd.set_option('display.max_columns', 20) #display all columns to ensure every thing looks right
pd.set_option('display.max_rows', None)
f_data.describe().T

,count,mean,std,min,25%,50%,75%,max
DR_1,13910.0,50435.066174,69844.785952,-16757.598600,6694.725950,19364.439350,63104.837125,670687.347700
|DR|_1,13910.0,6.638156,13.486391,0.088287,2.284843,3.871227,8.400619,1339.879283
EMAi0.001_1,13910.0,12.936688,17.610061,0.000100,1.633350,4.977123,17.189165,167.079751
EMAi0.01_1,13910.0,18.743953,24.899450,0.000100,2.386836,7.250892,26.411109,226.619457
EMAi0.1_1,13910.0,26.890695,38.107685,0.000100,4.967988,11.680725,34.843226,993.605306
EMAd0.001_1,13910.0,-9.158655,12.729206,-131.332873,-11.587169,-3.338700,-1.126897,-0.006941
EMAd0.01_1,13910.0,-14.402383,21.304606,-227.627758,-17.292559,-4.956917,-1.670327,22.201589
EMAd0.1_1,13910.0,-59.927598,131.017675,-1664.735576,-48.492764,-14.040088,-5.212213,115.273147
DR_2,13910.0,57340.104585,64045.265134,-16119.460900,13287.301875,37764.263200,70300.782575,502202.812500
|DR|_2,13910.0,6.648033,15.585780,0.185164,2.776693,4.734586,8.608522,1672.363221


All sensors seem to have a couple of outlier readings when comparing the max values to the mean and 75% values for all parameters except EMAd0.001 because there is approximately a 5x increase or change in sign.

However, this could be due to the fact that there supposedly was a run with a gas concentration value of 1000.


In [39]:
pd.set_option('display.max_rows', 20)
missing = pd.concat([f_data.isnull().sum(), 100 * f_data.isnull().mean()], axis=1)
missing.columns=['count','%']
missing.sort_values(by='count', ascending=False)

,count,%
DR_1,0,0.0
EMAi0.001_13,0,0.0
DR_13,0,0.0
EMAd0.1_12,0,0.0
EMAd0.01_12,0,0.0
...,...,...
EMAd0.01_5,0,0.0
EMAd0.001_5,0,0.0
EMAi0.1_5,0,0.0
EMAi0.01_5,0,0.0


There is no mising data in the data set, which is either very lucky or unlucky. If it is a complete data set and all values are correct then we are lucky. Alternatively, if missing values were filled in with a chosen value then we need to determine what value correlates to a missing data point.